<a href="https://colab.research.google.com/github/Andrian0s/ML4NLP1-2024-Tutorial-Notebooks/blob/main/exercises/ex1/ex1_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Teammates:
Yuxuan Wang 24-741-399
Leyi Hu 24-756-108
Jinglei Huang 23-746-217

# ML4NLP1
## Starting Point for Exercise 1, part II

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part II.

One of the goals of this exercise is o make you acquainted with **skorch**. You will probably need to consult the [documentation](https://skorch.readthedocs.io/en/stable/).

# Installing skorch and loading libraries

In [ ]:
pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 17.5 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

# Set seed for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

## Training a classifier and making predictions

In [ ]:
# Download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /content/x_train.txt
100% 64.1M/64.1M [00:00<00:00, 75.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /content/x_test.txt
100% 65.2M/65.2M [00:01<00:00, 37.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /content/y_train.txt
100% 480k/480k [00:00<00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /content/y_test.txt
100% 480k/480k [00:00<00:00, 148MB/s]


In [ ]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [ ]:
# Combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})
# Write train_df to csv with tab as separator
train_df.to_csv('train_df.csv', index=False, sep='\t')
# Comibne x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})
# Inspect the first 5 items in the train split
train_df.head()

,text,label
0,Klement Gottwaldi surnukeha palsameeriti ning ...,est
1,"Sebes, Joseph; Pereira Thomas (1961) (på eng)....",swe
2,भारतीय स्वातन्त्र्य आन्दोलन राष्ट्रीय एवम क्षे...,mai
3,"Après lo cort periòde d'establiment a Basilèa,...",oci
4,ถนนเจริญกรุง (อักษรโรมัน: Thanon Charoen Krung...,tha


### Data preparation

Prepare your dataset for this experiment using the same method as you did in part 1.

Get a subset of the train/test data that includes 20 languages. Include English, German, Dutch, Danish, Swedish, Norwegian, and Japanese, plus 13 additional languages of your choice based on the items in the list of labels.

Don't forget to encode your labels using the adjusted code snippet from part 1!


In [ ]:
# # TODO: Create your train/test subsets of languages
required_languages = ['eng', 'deu', 'nld', 'dan', 'swe', 'nob', 'jpn', 'zh-yue', 'zho']

all_languages = train_df['label'].value_counts()
additional_languages = []

for lang in all_languages.index:
    if lang not in required_languages and len(additional_languages) < 11:
        additional_languages.append(lang)

selected_languages = required_languages + additional_languages
print(f"Selected 20 languages: {selected_languages}")

# Create subsets
train_subset = train_df[train_df['label'].isin(selected_languages)].copy()
test_subset = test_df[test_df['label'].isin(selected_languages)].copy()

print(f"Original training size: {len(train_df)}")
print(f"Subset training size: {len(train_subset)}")
print(f"Original test size: {len(test_df)}")
print(f"Subset test size: {len(test_subset)}")

# Check if all labels in subset have the same count
print(f"\nChecking if dataset is balanced:")
print(f"All training languages have exactly 500 samples: {(train_subset['label'].value_counts() == 500).all()}")
print(f"All test languages have exactly 500 samples: {(test_subset['label'].value_counts() == 500).all()}")

# Update our working datasets
x_train_subset = train_subset['text'].tolist()
y_train_subset = train_subset['label'].tolist()
x_test_subset = test_subset['text'].tolist()
y_test_subset = test_subset['label'].tolist()

print(f"Training subset: {len(x_train_subset)} samples")
print(f"Test subset: {len(x_test_subset)} samples")

Selected 20 languages: ['eng', 'deu', 'nld', 'dan', 'swe', 'nob', 'jpn', 'zh-yue', 'zho', 'est', 'mai', 'oci', 'tha', 'orm', 'lim', 'guj', 'pnb', 'zea', 'krc', 'hat']
Original training size: 117500
Subset training size: 10000
Original test size: 117500
Subset test size: 10000

Checking if dataset is balanced:
All training languages have exactly 500 samples: True
All test languages have exactly 500 samples: True
Training subset: 10000 samples
Test subset: 10000 samples


In [ ]:
# TODO: With the following code, we wanted to ENCODE the labels, however, our cat was walking on the keyboard and some of it got changed. Can you fix it?
from sklearn.preprocessing import LabelEncoder

# Create and fit the label encoder on training labels
label_encoder = LabelEncoder()
label_encoder.fit(y_train_subset)

# Transform both training and test labels
y_train_encoded = label_encoder.transform(y_train_subset)
y_test_encoded = label_encoder.transform(y_test_subset)

print(f"Original labels: {y_train_subset[:5]}")
print(f"Encoded labels: {y_train_encoded[:5]}")
print(f"\nUnique encoded labels: {sorted(set(y_train_encoded))}")
print(f"Number of classes: {len(label_encoder.classes_)}")

Original labels: ['est', 'swe', 'mai', 'oci', 'tha']
Encoded labels: [ 3 15  9 12 16]

Unique encoded labels: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10), np.int64(11), np.int64(12), np.int64(13), np.int64(14), np.int64(15), np.int64(16), np.int64(17), np.int64(18), np.int64(19)]
Number of classes: 20


### Feature Extraction

In [ ]:
# First, we extract some simple features as input for the neural network
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True)
X = vectorizer.fit_transform(x_train_subset )

In [ ]:
# We need to change the datatype to make it play nice with pytorch
X = X.astype(np.float32)
y = y_train_encoded.astype(np.int64)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [ ]:
# TODO: In the following, you can find a small (almost) working example of a neural network.
# Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable. (Hint: the input and output sizes look a bit weird...)

class ClassifierModule(nn.Module):
    def __init__(
        self,
        input_size,
        num_classes,
        h1=200,
        h2=50,
        nonlin=F.relu,
        dropout=0.0,
    ):
        super(ClassifierModule, self).__init__()
        self.h1 = h1
        self.h2 = h2
        self.nonlin = nonlin

        self.dense0 = nn.Linear(input_size, h1)
        self.dense1 = nn.Linear(h1, h2)
        self.output = nn.Linear(h2, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.dropout(X)
        X = self.output(X)
        return X

In [ ]:
# Initalise the neural net classifier.
net = NeuralNetClassifier(
    ClassifierModule(
        input_size=X.shape[1],
        num_classes=len(label_encoder.classes_),
        h1=200,
        h2=50,
        nonlin=F.relu,
    ),
    max_epochs=30,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    device='cuda',  # comment this to train with CPU
)

In [ ]:
# Train the classifier
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.8522       0.1910        2.6813  1.2783
      2        2.5132       0.2960        2.2994  0.9180
      3        2.0469       0.3935        1.8550  0.6792
      4        1.6882       0.4145        1.6009  0.5732
      5        1.4627       0.4795        1.4199  0.6132
      6        1.2992       0.5535        1.2779  0.6202
      7        1.1755       0.5890        1.1765  0.6530
      8        1.0930       0.6130        1.1111  0.6516
      9        1.0406       0.6215        1.0704  0.5945
     10        1.0037       0.6180        1.0417  0.6318
     11        0.9752       0.6265        1.0208  0.6167
     12        0.9518       0.6240        1.0050  0.5889
     13        0.9325       0.6250        0.9919  0.5973
     14        0.9162       0.6285        0.9819  0.6031
     15        0.9025       0.6310        0.9729  0.6015
     16        0.8906       0.6

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=100, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=20, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  ),
)

In [ ]:
# Grid Search for
# - layer sizes
# - activation functions
# - regularizers
# - drop out
# - vectorizer parameters
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')

vectorizers = {
    'bigram_100': CountVectorizer(analyzer='char', ngram_range=(2, 2), max_features=100, binary=True),
    'trigram_150': CountVectorizer(analyzer='char', ngram_range=(3, 3), max_features=150, binary=True),
    'mixed_200': CountVectorizer(analyzer='char', ngram_range=(2, 3), max_features=200, binary=True)
}

results_summary = []

for vec_name, vectorizer in vectorizers.items():
    print(f"\n{'='*50}")
    print(f"Testing with vectorizer: {vec_name}")
    print(f"{'='*50}")

    # Extract features with current vectorizer
    X_current = vectorizer.fit_transform(x_train_subset)
    X_current = X_current.astype(np.float32)

    print(f"Feature dimensions: {X_current.shape}")

    # Define parameter grid with activation functions
    param_grid = {
        'module__h1': [200, 300],
        'module__h2': [50, 80],
        'module__nonlin': [F.relu, F.tanh, F.leaky_relu],
        'lr': [0.01, 0.1],
        'max_epochs': [15],
        'module__dropout': [0.0, 0.2],
    }

    grid_net = NeuralNetClassifier(
        module=ClassifierModule,
        module__input_size=X_current.shape[1],
        module__num_classes=len(label_encoder.classes_),
        module__h1=200,  # Default values
        module__h2=50,
        module__nonlin=F.relu,
        module__dropout=0.0,
        criterion=nn.CrossEntropyLoss(),
        device='cuda' if torch.cuda.is_available() else 'cpu',
        verbose=0,
    )

    print("Starting Grid Search...")

    grid_search = GridSearchCV(
        grid_net,
        param_grid,
        cv=3,
        scoring='accuracy',
        n_jobs=1,
        verbose=1
    )

    # Fit grid search
    grid_search.fit(X_current, y)

    # Store results
    result = {
        'vectorizer': vec_name,
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'feature_count': X_current.shape[1]
    }
    results_summary.append(result)

    print(f"\nBest parameters for {vec_name}: {grid_search.best_params_}")
    print(f"Best CV score: {grid_search.best_score_:.4f}")



Testing with vectorizer: bigram_100
Feature dimensions: (10000, 100)
Starting Grid Search...
Fitting 3 folds for each of 48 candidates, totalling 144 fits

Best parameters for bigram_100: {'lr': 0.1, 'max_epochs': 15, 'module__dropout': 0.0, 'module__h1': 300, 'module__h2': 80, 'module__nonlin': <function tanh at 0x7bf8f360b2e0>}
Best CV score: 0.6268

Testing with vectorizer: trigram_150
Feature dimensions: (10000, 150)
Starting Grid Search...
Fitting 3 folds for each of 48 candidates, totalling 144 fits

Best parameters for trigram_150: {'lr': 0.1, 'max_epochs': 15, 'module__dropout': 0.0, 'module__h1': 300, 'module__h2': 80, 'module__nonlin': <function tanh at 0x7bf8f360b2e0>}
Best CV score: 0.5886

Testing with vectorizer: mixed_200
Feature dimensions: (10000, 200)
Starting Grid Search...
Fitting 3 folds for each of 48 candidates, totalling 144 fits

Best parameters for mixed_200: {'lr': 0.1, 'max_epochs': 15, 'module__dropout': 0.0, 'module__h1': 300, 'module__h2': 80, 'module__n

In [ ]:
# Best hyperparameters specifically
# Best parameters for mixed_200: {'lr': 0.1, 'max_epochs': 15, 'module__dropout': 0.0, 'module__h1': 300, 'module__h2': 80
# Train the model with best hyperparameters with more epochs
h1 = 300
h2 = 80
best_vectorizer = CountVectorizer(analyzer='char', ngram_range=(2, 3), max_features=200, binary=True)
nonlin = F.tanh
dropout = 0.0
num_epochs = 40
lr=0.1


X_train_best = best_vectorizer.fit_transform(x_train_subset)
X_train_best = X_train_best.astype(np.float32)

X_test_best = best_vectorizer.transform(x_test_subset)
X_test_best = X_test_best.astype(np.float32)

# Create model with the best hyperparameters found
best_model = NeuralNetClassifier(
    ClassifierModule(
        input_size=X_train_best.shape[1],
        num_classes=len(label_encoder.classes_),
        h1=h1,
        h2=h2,
        nonlin=nonlin,
        dropout=dropout,
    ),
    max_epochs=num_epochs,
    criterion=nn.CrossEntropyLoss(),
    lr=lr,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    verbose=1,
)

# Train the model with best hyperparameters
print("Training model with best hyperparameters...")
best_model.fit(X_train_best, y)

# Make predictions on test set
test_predictions = best_model.predict(X_test_best)
test_accuracy = (test_predictions == y_test_encoded).mean()

print(f"\nTest set accuracy with best parameters: {test_accuracy:.4f}")

Training model with best hyperparameters...
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.5996       0.5305        2.1657  0.5719
      2        1.7704       0.5510        1.4552  0.5978
      3        1.3046       0.6325        1.1872  0.5935
      4        1.1019       0.6730        1.0410  0.5802
      5        0.9761       0.6815        0.9515  0.5964
      6        0.8960       0.6935        0.8952  0.5892
      7        0.8405       0.7120        0.8549  0.5888
      8        0.7977       0.7165        0.8230  0.6200
      9        0.7627       0.7220        0.7969  0.5835
     10        0.7337       0.7265        0.7757  0.6121
     11        0.7098       0.7275        0.7587  0.6190
     12        0.6899       0.7300        0.7452  0.5951
     13        0.6734       0.7315        0.7344  0.5739
     14        0.6594       0.7355        0.7258  0.5950
     15        0.6474       0.7365        0.

Note, you can also use `GridSearchCV` with `skorch`, but be aware that training a neural network takes much more time.

Play around with 5 different sets of hyperparameters. For example, consider some of the following:

- layer sizes
- activation functions
- regularizers
- early stopping
- vectorizer parameters

Report your best hyperparameter combination. \\
📝❓ What is the effect of your modifcations on validation performance? Discuss potential reasons.

☝ Note, during model development, if you run into the infamous CUDA out-of-memory (OOM) error, try clearing the GPU memory either with `torch.cuda.empty_cache()` or restarting the runtime.


---
Lab report

1. Best hyperparameters
   - layer sizes: h1 = 300, h2 = 80
   - activation functions: Tanh
   - early stopping: 40 epochs
   - vectorizer parameter: mixed 2-garam and 3-gram with 200 max features
   - learning rate: 0.1
   - no drop out

📝❓ What is the effect of your modifcations on validation performance? Discuss potential reasons.
1. Increaing the hidden states (h1,h2) can increase the performance, which is reasonable, since more hidden states can encode more information into latent space. However too much hidden states can lead to overfitting.
2. Too small lr results in too slow converge, too large lr leads to unstable learning
3. Intuitively adding the drop as a regularizer should have better performance, but suprisingly the best fit of grid search has 0.0 drop out.